In [ ]:
# 操作 browser 的 API
import csv
import random
import re

# 多執行緒
import threading

# 強制等待 (執行期間休息一下)
from time import sleep, time

import keyboard

# listening
import pyautogui
from IPython.display import clear_output
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# send_keys模擬鍵盤輸入
from selenium.webdriver.common.keys import Keys

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait


def initialize_browser():
    # 啟動瀏覽器工具的選項
    my_options = webdriver.ChromeOptions()
    # my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
    my_options.add_argument("--start-maximized")  # 最大化視窗
    my_options.add_argument("--incognito")  # 開啟無痕模式
    my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
    my_options.add_argument("--disable-notifications")  # 取消 chrome 推播通知
    my_options.add_argument("--lang=zh-TW")  # 設定為正體中文

    # 使用 Chrome 的 WebDriver
    driver = webdriver.Chrome(options=my_options)

    return driver


def scraping(driver):
    with open(r"./newtaipei_park.txt", "r", encoding="utf-8") as file:
        dist_list = file.readlines()
        dataset = [["name", "class", "address", "latitude", "longitude"]]

        # 前往google_map網站
        driver.get("https://www.google.com/maps?authuser=0")

        count = 0
        
        for search_name in dist_list:
            search_name = search_name.replace("\n", "")

            # 尋找網頁中的搜尋框
            input_block = driver.find_element(
                By.CSS_SELECTOR, "input.searchboxinput.xiQnY"
            )
            print(f"{search_name}-starting...")
            
            if count != 0:
                input_block.send_keys(Keys.CONTROL,"a")
                sleep(0.2)
                input_block.send_keys(Keys.DELETE)
                sleep(0.2)

            count += 1
            
            # 在搜尋框中輸入文字
            input_block.send_keys(f"{search_name}")
            sleep(0.1)
            input_block.send_keys(Keys.ENTER)
            sleep(0.5)
            

            try:
                # 取得google_map基本內容
                element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.lMbq3e"))
                )

                sleep(2)

                # 車站名稱
                name = element.find_element(By.CSS_SELECTOR, "h1.DUwDvf.lfPIob")

                # 設施種類
                class_ = element.find_element(By.CSS_SELECTOR, "button.DkEaL ")

                # 車站地址
                address = driver.find_element(
                    By.CSS_SELECTOR, "div.Io6YTe.fontBodyMedium.kR99db"
                )

                regex_address = r"[^0-9].*"  # 106台北市大安區復興南路二段206號 => 台北市大安區復興南路二段206號
                search_address = re.search(regex_address, address.text)

                # coordinate(經緯度)
                link = driver.current_url  # 當前網頁網址
                regex_coordinate = (
                    r".\w.\w+!4d.\w+.\w+"  # link => 25.0266645!4d121.5431999
                )
                search_coordinate = re.search(regex_coordinate, link)
                # 切分成緯度、經度
                coordinate = search_coordinate[0].split(
                    "!4d"
                )  # coordinate:座標(經緯度)

                dataset.append(
                    [
                        name.text,
                        class_.text,
                        search_address[0],
                        coordinate[0],
                        coordinate[1]
                    ]
                )
                print(123)
                

            except:
                try:
                    search_block = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "a.hfpxzc"))
                    )

                    search_block.click()

                    element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.lMbq3e"))
                    )

                    sleep(2)

                    # 車站名稱
                    name = element.find_element(By.CSS_SELECTOR, "h1.DUwDvf.lfPIob")

                    # 設施種類
                    class_ = element.find_element(By.CSS_SELECTOR, "button.DkEaL ")

                    # 車站地址
                    address = driver.find_element(
                        By.CSS_SELECTOR, "div.Io6YTe.fontBodyMedium.kR99db"
                    )

                    regex_address = r"[^0-9].*"  # 106台北市大安區復興南路二段206號 => 台北市大安區復興南路二段206號
                    search_address = re.search(regex_address, address.text)

                    # coordinate(經緯度)
                    link = driver.current_url  # 當前網頁網址
                    regex_coordinate = (
                        r".\w.\w+!4d.\w+.\w+"  # link => 25.0266645!4d121.5431999
                    )
                    search_coordinate = re.search(regex_coordinate, link)
                    # 切分成緯度、經度
                    coordinate = search_coordinate[0].split(
                        "!4d"
                    )  # coordinate:座標(經緯度)

                    dataset.append(
                        [
                            name.text,
                            class_.text,
                            search_address[0],
                            coordinate[0],
                            coordinate[1]
                        ]
                    )
                    print(1234)
                    
                except:
                    print(search_name + "null")
                    dataset.append(
                        [
                            search_name,
                            "null",
                            "null",
                            "null",
                            "null"
                        ])

                    
                    continue
        return dataset


def to_csv(dataset):
    # 將 list 存成 csv
    with open(f"./new_taipei_park.csv", "w", newline="", encoding="utf-8-sig") as file:
        w = csv.writer(file)
        # 寫入二維表格
        w.writerows(dataset)
    #    print(f'{search_name}-ending...')


def listening():
    global switch

    while switch:
        # 睡一下
        sleep(0.01)

        # 自動清除此格的文字輸出
        clear_output(wait=True)

        try:
            # 取得 Box 物件
            btn_reload = pyautogui.locateOnScreen(
                "./reload.png",
                confidence=0.9,  # opencv信心指數，用 1 的話圖片一點灰塵污漬都不能有
            )

            btn_reload_point = pyautogui.center(btn_reload)

            # 按下 重新載入
            pyautogui.click(btn_reload_point.x, btn_reload_point.y)
        except:
            pass


if __name__ == "__main__":
    driver = initialize_browser()
    switch = True
    listening_thread = threading.Thread(target=listening)  # 建立listening執行緒
    listening_thread.start()  # 啟動執行緒

    dataset = scraping(driver)
    to_csv(dataset)

    switch = False  # 爬蟲結束，結束迴圈
    listening_thread.join()  # 结束執行緒

    driver.quit()


In [ ]:
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep


# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)

# 前往北捷網站
driver.get('https://www.metro.taipei/cp.aspx?n=91974F2B13D997F1')


# 切換到指定的frame
driver.switch_to.frame("route")

# 站名標籤
all_station = driver.find_elements(
                By.CSS_SELECTOR,
                'optgroup > option'
            )

# 取得站名
for station in all_station:
    print(station.text)

In [5]:
!pip install geocoder

     ---------------------------------------- 0.0/98.6 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/98.6 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/98.6 kB ? eta -:--:--
     ----------- -------------------------- 30.7/98.6 kB 220.2 kB/s eta 0:00:01
     --------------------------- ---------- 71.7/98.6 kB 359.3 kB/s eta 0:00:01
     -------------------------------------- 98.6/98.6 kB 472.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/491.3 kB ? eta -:--:--
   --------- ------------------------------ 112.6/491.3 kB 3.3 MB/s eta 0:00:01
   --------------- ------------------------ 194.6/491.3 kB 2.0 MB/s eta 0:00:01
   ------------------- -------------------- 235.5/491.3 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------  481.3/491.3 kB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 491.3/491.3 kB 2.6 MB/s eta 0:00:00


In [7]:
import geocoder
coordinates = geocoder.arcgis(f'台北市信義區信義路五段7號').latlng
coo

[25.03372000764763, 121.56480998805017]

In [30]:
import geocoder
from shapely.geometry import Point

address = "桃園市桃園區桃鶯路110號1樓"
coordinates = geocoder.arcgis(address).latlng


# 给定的经纬度
longitude = coordinates[1]
latitude = coordinates[0]


# 创建中心点
center_point = Point(longitude, latitude)

In [31]:
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import sjoin


# 加载饮料店数据
# 给定的经纬度
longitude = coordinates[1]
latitude = coordinates[0]


# 创建中心点
center_point = Point(longitude, latitude)

center_gdf = gpd.GeoDataFrame(geometry=[center_point])

# 加载学校数据
schools_df = gpd.read_file('./雙北學校.csv')
schools_gdf = gpd.GeoDataFrame(
    schools_df, 
    geometry=[Point(xy) for xy in zip(schools_df.longitude, schools_df.latitude)]
)

# 设置坐标参考系统为 WGS84
center_gdf.crs = "epsg:4326" # epsg:3826
schools_gdf.crs = "epsg:4326"

# drink_shops_df

In [32]:
# 转换坐标系统到 Web Mercator (epsg:3826) 以计算以米为单位的距离
center_gdf = center_gdf.to_crs(epsg=3826)
schools_gdf = schools_gdf.to_crs(epsg=3826)

# 为每个饮料店创建方圆 1 公里的缓冲区
center_gdf['buffer'] = center_gdf.geometry.buffer(1000, resolution=99)  # 1000 米



In [33]:
# 假设这是添加标识符的步骤
center_gdf['shop_id'] = range(len(center_gdf))

# 创建缓冲区 GeoDataFrame，并包括 shop_id
buffers_gdf = gpd.GeoDataFrame(center_gdf[['shop_id', 'geometry']].copy())
buffers_gdf['geometry'] = buffers_gdf.geometry.buffer(1000)  # 创建缓冲区


In [35]:
# 执行空间连接
joined_gdf = sjoin(buffers_gdf, schools_gdf, how="inner", predicate='contains')

# 统计每个缓冲区（即每个饮料店）内的学校数量
school_counts = joined_gdf.groupby('shop_id').size()

# 为了确保即使是点数为 0 的多边形也能被统计
# 我们需要将计数结果与原始的多边形 GeoDataFrame 进行合并
buffers_gdf['points_count'] = buffers_gdf.index.map(school_counts).fillna(0)

# print(school_counts)

buffers_gdf['points_count']



0    0.0
Name: points_count, dtype: float64